# Пробуем реализовать задачу через TabPFN
https://github.com/PriorLabs/TabPFN?tab=readme-ov-file

In [1]:
# Install sklearn
!pip install scikit-learn==1.5.2

# Install TabPFN API Client
!pip install tabpfn-client

# TabPFN Extensions installs optional functionalities around the TabPFN model
# These include post-hoc ensembles, interpretability tools, and more
!git clone https://github.com/PriorLabs/tabpfn-extensions
!pip install -e tabpfn-extensions
!pip install tabpfn # TabPFN, currently required by tabpfn-extensions, but not basic usage


# Install Baselines
!pip install catboost xgboost

# Install example datasets
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 2.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.8.1 requires scikit-learn>=1.6.0, but you have scikit-learn 1.5.2 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
  DEPRECATION: Wheel filename 'password_strength-0.0.1_1-py2-none-any.whl' is not correctly normalised. Future versions of pip will raise the following error:
  Invalid wheel filename (invalid version): password_strength-0.0.1_1-py2-none-any
  
   pip 25.1 will enforce this behaviour change. A possible replacement is to rename the wheel 

In [4]:
import os

# Setup Imports
import pandas as pd
import numpy as np

from sklearn.datasets import load_breast_cancer, load_diabetes, load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import (
    accuracy_score,
    mean_absolute_error,
    mean_squared_error,
    root_mean_squared_error,
    r2_score,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.inspection import DecisionBoundaryDisplay

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from IPython.display import display, Markdown, Latex

# Baseline Imports
from xgboost import XGBClassifier, XGBRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from catboost import CatBoostClassifier, CatBoostRegressor


from tabpfn_extensions.post_hoc_ensembles.sklearn_interface import AutoTabPFNClassifier, AutoTabPFNRegressor

In [5]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

import joblib
import numpy as np
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn
import pandas as pd
import seaborn as sns
import warnings

from catboost import CatBoostRegressor
from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Вместо одного фиксированного разбиения на train/test используем стабильную стратегию кросс-валидации.
# Используем тут Cross-validation, потому что:
# 	•	нужно надёжно сравнить несколько разных моделей или гиперпараметров и понять, какая модель стабильнее и лучше в целом.
# 	•	хотим избежать случайных удач или провалов, связанных с конкретным разбиением на train/test.
# 	•	выбираем модель или гиперпараметры, которые потом будешь использовать для финального сабмишна на Kaggle.
# Делаем эту оценку, чтобы в дальнейших блокнотах-улучшениях сравнивать более корректно.
from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score, train_test_split

# Используем IterativeImputer:
# 	•	Он итеративно заполняет все пропуски сразу.
# 	•	Работает одновременно со всеми признаками, учитывая связи между ними.
# 	•	Не требует ручного управления порядком заполнения.
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from utils.data_manager import DataManager
from utils.model_manager import ModelManager
from utils.syth_generator_gaussian import CombinedSyntheticGenerator

In [6]:
# TabPFN Imports
from tabpfn_client import init, TabPFNClassifier, TabPFNRegressor

init()

  Welcome Back! Found existing access token, reusing it for authentication.


In [7]:
# --- Глобально включаем вывод Pandas для всех трансформеров ---
# (Можно применять и к отдельным трансформерам/пайплайнам .set_output(transform="pandas"))
set_config(transform_output = "pandas")

In [8]:
dm = DataManager()
mm = ModelManager()

# Отключаем автологгирование, чтобы использовать ручное
mlflow.sklearn.autolog(disable=True)
warnings.filterwarnings("ignore", module="mlflow")  # Игнорируем предупреждения MLflow


In [9]:
RANDOM_STATE = 42
N_FOLDS = 5  # Например, 5 или 10

## 1. Загрузка данных

In [10]:
data_path = 'data/home-data-for-ml-course'
train_data = pd.read_csv(data_path + '/train.csv')
test_data = pd.read_csv(data_path + '/test.csv')
train_data.shape

(1460, 81)

In [11]:
# # Пробуем нагенерить синтетики
# # Инициализация и обучение генератора
# generator = CombinedSyntheticGenerator(use_gaussian_copula=True)
# generator.fit(train_data)
# 
# # Генерация синтетических данных
# synthetic_df = generator.generate(count=500)
# 
# # Сохранение результатов
# synthetic_df.to_csv('synthetic_data3.csv', index=False)
# 
# sdf = pd.read_csv('synthetic_data3.csv')
# # Объединяем с реальными данными
# train_data = pd.concat([train_data, sdf], ignore_index=True)
# train_data.shape

## 2. Предобработка данных 

In [12]:
# Определение колонок для удаления
intuitively_bad_features = [
    'LotShape',  # Общая форма участка
    'LandContour',  # Рельеф участка
    'LotConfig',  # Конфигурация участка
    'LandSlope',  # Уклон участка
    'MiscFeature',
    'MiscVal',
]
bad_columns = dm.get_all_nan_cols(train_data)
bad_columns.append('Id')
bad_columns.extend(intuitively_bad_features)

In [13]:
# Разделение на X / y
X, y = dm.split_data_set_to_x_y(train_data, 'SalePrice')
print(X.shape, y.shape)
X_test = test_data.copy()
print(X_test.shape)

(1460, 80) (1460,)
(1459, 80)


In [14]:
X.drop(columns=bad_columns, inplace=True)
X_test.drop(columns=bad_columns, inplace=True)

In [15]:
def make_feature_eng_great_again(train_X_in, test_X_in):
    """Хелпер, который создает фичи, логарифмирует и выравнивает колонки."""
    # Работаем с копиями, чтобы не изменять оригинальные X, X_test вне функции
    train_X = train_X_in.copy()
    test_X = test_X_in.copy()

    # Словарь качественных признаков
    quality_dict = {'Ex': 5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, np.nan:0}

    def create_features(df):
        # Interactions (с проверкой на наличие колонок)
        if 'Neighborhood' in df.columns and 'MSZoning' in df.columns:
            df['Neighborhood_Zoning'] = df['Neighborhood'].astype(str) + '_' + df['MSZoning'].astype(str)
        if 'SaleType' in df.columns and 'SaleCondition' in df.columns:
            df['SaleType_Condition'] = df['SaleType'].astype(str) + '_' + df['SaleCondition'].astype(str)

        # Quality Score
        df['TotalQualScore'] = 0
        quality_cols = ['ExterQual', 'KitchenQual', 'BsmtQual', 'HeatingQC', 'GarageQual', 'FireplaceQu']
        for col in quality_cols:
            if col in df.columns:
                 df['TotalQualScore'] += df[col].map(quality_dict).fillna(0)

        # Porch/Deck Area and Flags
        df['PorchDeckArea'] = 0
        porch_cols = ['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']
        for col in porch_cols:
             if col in df.columns:
                df['PorchDeckArea'] += df[col].fillna(0)

        if 'Fireplaces' in df.columns:
            df['HasFireplace'] = (df['Fireplaces'] > 0).astype(int)
        if 'GarageType' in df.columns:
            df['HasGarage'] = (~df['GarageType'].isna()).astype(int)
        if 'Fence' in df.columns:
            df['HasFence'] = (~df['Fence'].isna()).astype(int)
        df['HasPorchDeck'] = (df['PorchDeckArea'] > 0).astype(int)

        return df

    def log_features(df, cols_to_log_list):  # Принимает СПИСОК колонок
        print(f"Applying log1p to: {cols_to_log_list}")
        for col_name in cols_to_log_list:
            if col_name in df.columns:
                # Добавим проверку на отрицательные значения перед логарифмированием
                if (df[col_name] < 0).any():
                     print(f"Warning: Column {col_name} contains negative values. Skipping log1p.")
                else:
                    df[col_name] = np.log1p(df[col_name])
            else:
                print(f"Warning: Column {col_name} not found in DF during log transform.")
        return df

    # 1. Создаем фичи
    train_X = create_features(train_X)
    test_X = create_features(test_X)
    print("Features created.")

    # 2. Определяем колонки для логарифмирования (ТОЛЬКО по трейну)
    numeric_cols = train_X.select_dtypes(include=np.number).columns
    skew_values = train_X[numeric_cols].skew()
    # Используем .index.tolist() чтобы получить список имен
    cols_to_log_list = skew_values[skew_values > 1].index.tolist()
    print(f"Columns identified for logging: {cols_to_log_list}")

    # # 3. Логарифмируем (используя ОДИН и тот же список)
    # train_X = log_features(train_X, cols_to_log_list)
    # test_X = log_features(test_X, cols_to_log_list)
    # print("Log transform applied.")

    # 4. Согласуем и сортируем колонки ПОСЛЕ всех манипуляций
    final_feature_cols = sorted(train_X.columns.tolist()) # Сортируем для стабильности
    train_X = train_X[final_feature_cols]
    test_X = test_X.reindex(columns=final_feature_cols, fill_value=0)
    print("Columns aligned and sorted.")

    return train_X, test_X

In [16]:
# Вызываем функцию с правильными данными (X, X_test)
X, X_test = make_feature_eng_great_again(X, X_test)

print("\nProcessing complete. Final shapes:")
print(f"X_processed: {X.shape}")
print(f"X_test_processed: {X_test.shape}")
print("\nExample processed X:")
print(X.head())

Features created.
Columns identified for logging: ['MSSubClass', 'LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', '1stFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtHalfBath', 'KitchenAbvGr', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PorchDeckArea', 'HasFence']
Columns aligned and sorted.

Processing complete. Final shapes:
X_processed: (1460, 81)
X_test_processed: (1459, 81)

Example processed X:
   1stFlrSF  2ndFlrSF  3SsnPorch Alley  BedroomAbvGr BldgType BsmtCond  \
0       856       854          0   NaN             3     1Fam       TA   
1      1262         0          0   NaN             3     1Fam       TA   
2       920       866          0   NaN             3     1Fam       TA   
3       961       756          0   NaN             3     1Fam       Gd   
4      1145      1053          0   NaN             4     1Fam       TA   

  BsmtExposure  BsmtFinSF1  BsmtFinSF2  ... ScreenPorch Street  TotRmsAbvGrd  \


In [17]:
# Получение числовых колонок
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns
# Получение нечисловых колонок (всех остальных)
non_numeric_columns = X.select_dtypes(exclude=['float64', 'int64']).columns

## 3. Обучаем модель с CV и корректируя данные

#### Нормализация данных через ColumnTransformer и Pipeline
В данном кейсе мы реализуем Заполнение числовых пропусков с помощью модели (Predictive imputation). Т.е. то, что пропущено в числовых признаках - будем заполнять не медианой или средним, а будем обучать модель, которая будет предсказывать пропуски (IterativeImputer + RandomForestRegressor)

In [18]:
# Создаем preprocessor с разными трансформерами для разных типов данных
# Числовые данные пропущенные предсказываем с помощью модели RandomForestRegressor

# Пайплайн для числовых признаков (итеративное заполнение)
numeric_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(
        estimator=RandomForestRegressor(n_estimators=50, random_state=RANDOM_STATE),
        max_iter=10,
        random_state=RANDOM_STATE
    )),  # Дает примерно +100 прирост качества vs mean/median
    # ('imputer', SimpleImputer(strategy='mean')),
    # ('scaler', StandardScaler())  # разницы не дает 
])

# Пайплайн для категориальных признаков (заполнение частым значением и кодирование)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# --- Объединяем препроцессоры ---
preprocessor = ColumnTransformer(
    transformers=[
        # Применяем к исходным числовым колонкам
        ('num', numeric_transformer, numeric_columns),
        # Применяем к исходным категориальным колонкам
        ('cat', categorical_transformer, non_numeric_columns)
    ],
    remainder='drop',   # 'passthrough' сохранит полиномиальные и другие колонки, которые не были ни числовыми, ни категориальными ИЗНАЧАЛЬНО
    verbose_feature_names_out=False  # Чтобы имена колонок не менялись на 'num__colname' и т.д.
)

default_params = {
    'iterations': 1000, 
    'learning_rate': 0.05, 
    'depth': 6, 
    'loss_function': 'RMSE', 
    'verbose': 0, 
    'random_seed': RANDOM_STATE,
    'early_stopping_rounds': 100  # ?
}
catboost_params = default_params

# Включить Early Stopping очень рекомендуется в параметрах. Это позволит модели на каждом фолде останавливаться тогда, когда метрика на валидационной части этого фолда (X_val, y_val_log) перестает улучшаться. Это самый надежный способ подобрать оптимальное число итераций для каждого фолда и избежать переобучения

# --- Финальный пайплайн ---
# Мы будем использовать preprocessor и модель отдельно в цикле CV
# для корректной работы early stopping с пайплайном sklearn. Поэтому нам финальный пайплайн - не нужен.
# final_pipeline = Pipeline([
#     ('preprocessing', preprocessor),
#     ('model', CatBoostRegressor(**catboost_params))
# ])

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train.shape, X_test.shape

((1168, 81), (292, 81))

In [20]:
# Train and evaluate TabPFN
reg = TabPFNRegressor(random_state=42)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

score = root_mean_squared_error(np.log(y_test), np.log(y_pred))
print(f"TabPFN RMSLE: {score:.4f}")

Processing: 100%|██████████| [00:05<00:00]

TabPFN RMSLE: 0.1214


In [21]:
# К чему стремимся из catboost итерации
# Mean CV RMSE: 0.1258 +/- 0.0228
# OOF RMSE: 0.1278
# Mean best iteration: 1450


In [21]:
final_model_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('model', TabPFNRegressor(random_state=42))
])
final_model_pipeline.fit(X, y)
print("Final model trained.")

Final model trained.


In [28]:
# Предсказание на тест данных для сабмишна
final_test_pred_single_model = reg.predict(X_test)
submission_single = pd.DataFrame({'Id': test_data['Id'], 'SalePrice': final_test_pred_single_model})
submission_single.to_csv('submission_final_tabFPN_version.csv', index=False)
submission_single.head()

RuntimeError: Fail to call predict with error: {'detail': "API Daily Limit Reached. Your account's current limit is 5000000 table cell predictions per day. This limit resets daily at 00:00:00 UTC. You can make your next API request after 2025-04-07 00:00:00 UTC. To learn more about usage limits or request an increase, please visit: https://priorlabs.ai/api-usage-limit/"}